In [1]:
from bs4 import BeautifulSoup, Tag, NavigableString
import pandas as pd
import requests
import time
import json

In [1]:
from MALfunctions import malscrape

In [2]:
testlist = ['https://myanimelist.net/anime/1/Cowboy_Bebop', 
           'https://myanimelist.net/anime/5/Cowboy_Bebop__Tengoku_no_Tobira']

In [3]:
x = malscrape(testlist)

Processed anime [key = 1], 1/2, 50.00%: in 2.6902 seconds.
Processed anime [key = 5], 2/2, 100.00%: in 2.0012 seconds.
Scraping Complete


In [6]:
x.mallist

,Title,English,Japanese,Type,Episodes,Status,Aired,Premiered,Broadcast,Producers,...,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,Synonyms,ScoreExtract,ScoreRaters
key,,,,,,,,,,,,,,,,,,,,,
1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,TV,26,Finished Airing,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Saturdays at 01:00 (JST),Bandai Visual,...,24 min. per ep.,R - 17+ (violence & profanity),"8.811(scored by444,237users)1\n indicat...",#2522\n based on the top anime page. Please...,#38,884814,47575,NaN,8.811,444237
5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,Movie,1,Finished Airing,"Sep 1, 2001",NaN,NaN,"Sunrise,Bandai Visual",...,1 hr. 55 min.,R - 17+ (violence & profanity),"8.401(scored by128,760users)1\n indicat...",#15722\n based on the top anime page. Pleas...,#468,215121,881,Cowboy Bebop: Knockin' on Heaven's Door,8.401,128760


In [7]:
x.sequelkeys

,PrimaryKey,RelationType,RelationKey
2,1,Side story,5
3,1,Side story,17205
4,1,Summary,4037
5,5,Parent story,1


In [12]:
y = x.sequelkeys

In [13]:
y = y[y.RelationType != 'Adaptation']

In [14]:
sequelkeys

[]

In [9]:
%whos

Variable    Type        Data/Info
---------------------------------
malscrape   function    <function malscrape at 0x000001BC069A29D8>
testlist    list        n=2


In [3]:
def malkey(x, aslist = True):
    """returns an array of primary key[s] from url string[s]"""

    if isinstance(x, str):
        if aslist:
            return [int(s) for s in x.split('/') if s.isdigit()]
        else:
            return [int(s) for s in x.split('/') if s.isdigit()][0]
    
    elif isinstance(x, list):
        if len(x) > 1:
            return [malkey(y, aslist = False) for y in x]  
        else:
            return malkey(x)
    
    elif isinstance(x, Tag):
        if len(x) == 1:
            if isinstance(x.contents[0], Tag):
                # returning ['href'] gives a string, so use aslist argument
                if aslist:
                    return malkey(x.contents[0]['href'], aslist = True)
                else:
                    return malkey(x.contents[0]['href'], aslist = False)
            elif isinstance(x.contents[0], NavigableString):
                return malkey(x['href'], aslist = False)
            else:
                raise Exception('Detected neither Tag nor NavigableString')
        else:
            return [malkey(y, aslist = False) for y in x if isinstance(y, Tag) and len(y) > 0]
                
    else:
        raise Exception('Detected neither str nor list of str')

In [133]:
x

<td class="borderClass" width="100%"><a href="/anime/33909/Marine_Dreamin">Marine Dreamin'</a></td>

str

33909

In [109]:
malkey(y)

[46, 68]

In [3]:
with open('urllist.txt', 'r', encoding='utf8') as filehandle:  
    animelist = json.load(filehandle)

In [12]:
maldf = pd.read_json(r'maldf.json', orient='index')
maldf.sort_index(inplace=True)

In [13]:
maldf.columns

Index(['Aired', 'Broadcast', 'Duration', 'English', 'Episodes', 'Favorites',
       'Genres', 'Japanese', 'Licensors', 'Members', 'Popularity', 'Premiered',
       'Producers', 'Ranked', 'Rating', 'Score', 'ScoreExtract', 'ScoreRaters',
       'Source', 'Status', 'Studios', 'Synonyms', 'Title', 'Type'],
      dtype='object')

In [6]:
inserturl = 'https://myanimelist.net/anime/39595/Megazone_23_XI'

In [7]:
inserturl in animelist

True

In [69]:
malkey(inserturl)

39595

In [71]:
insertindex = len([i for i in animelist if malkey(i) < malkey(inserturl)])
insertindex

15684

In [72]:
animelist.insert(insertindex, inserturl)

In [77]:
animelist[insertindex-1]

'https://myanimelist.net/anime/39591/Persona_5_the_Animation_Specials'

In [78]:
with open('urllist.txt', 'w', encoding='utf8') as filehandle:  
    json.dump(animelist, filehandle)

## maldf update

In [49]:
def tagstringextract(x):
    """extract string from NavigableString"""
    if isinstance(x, NavigableString):
        return x.lstrip().rstrip()
    elif isinstance(x, Tag):
        return x.get_text().lstrip().rstrip()
    elif isinstance(x, list):
        return ''.join([tagstringextract(a) for a in x])
    else:
        raise Exception('Detected neither Tag nor NavigableString')
        
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [77]:
mallist = []
failedlist = []
sequelkeys = []
maxindex = len(urllist)
minlooptime = 2
failedcount = 0

NameError: name 'urllist' is not defined

In [60]:
urllist = [inserturl]
urllist

['https://myanimelist.net/anime/39595/Megazone_23_XI']

In [61]:
for idx, url in enumerate(urllist):
    # start timer
    timestart = time.time()
    
    try:
        page = requests.get(url,timeout=5)
        page.raise_for_status()
    except requests.exceptions.RequestException as e:
        failedcount += 1 
        print(e)
        print('Current Failed Count: ' + str(failedcount))
        failedlist.append(url)
        
        if time.time() - timestart < minlooptime:
            time.sleep(minlooptime - (time.time() - timestart))
        continue

    
    # get primary key, start new row
    animekey = malkey(url)
    newrow = {'key': animekey}
    
    # create soup
    anime = BeautifulSoup(page.content, 'html.parser')
    
    # get title
    title = {'Title': anime.select('.h1 span')[0].string}
    newrow.update(title)
    
    # get sidebar information
    sidebar = [a for a in 
               [[x for x in entry if x != '\n'] for entry in anime.select('.js-scrollfix-bottom div')]
               if len(a) > 1 and a[0].name == 'span']
    sidebardict = dict([entry[0].get_text(), tagstringextract(entry[1:])] for entry in sidebar)
    newrow.update(sidebardict)
    
    # add to global list
    mallist.append(newrow)
    
    # related
    relatedanime = anime.select('.anime_detail_related_anime .borderClass')
    relatedlist = [[animekey, alpha[0].text, y] for alpha in chunks(relatedanime, 2) for y in malkey(alpha[1])]
    
    sequelkeys += relatedlist
    
    # pause if too fast
    if time.time() - timestart < minlooptime:
        time.sleep(minlooptime - (time.time() - timestart))
    print('Processed anime [key = ' + str(animekey) + '], ' 
          + '%d' % (idx+1) + '/' + str(maxindex) + ', ' + '%.2f' % (100*(idx+1)/maxindex) + '%: in ' 
          + '%.4f' % (time.time() - timestart) + ' seconds.')

Processed anime [key = 39595], 1/1, 100.00%: in 2.0006 seconds.


In [15]:
mallist

[{'key': 39595,
  'Title': 'Megazone 23 XI',
  'Japanese:': 'メガゾーン23 XI',
  'Type:': 'TV',
  'Episodes:': 'Unknown',
  'Status:': 'Not yet aired',
  'Aired:': 'Not available',
  'Premiered:': '?',
  'Broadcast:': 'Unknown',
  'Producers:': 'None found,add some',
  'Licensors:': 'None found,add some',
  'Studios:': 'None found,add some',
  'Source:': 'Original',
  'Genres:': 'Action,Mecha,Music,Mystery,Romance,Sci-Fi',
  'Duration:': 'Unknown',
  'Rating:': 'None',
  'Score:': "N/A1(scored by0users)1\n        This score is not weighted. Please note that 'Not yet aired' titles are excluded.",
  'Ranked:': "#022\n    based on the top anime page. Please note that 'Not yet aired' and 'R18+' titles are excluded.",
  'Popularity:': '#0',
  'Members:': '0',
  'Favorites:': '0'}]

In [17]:
maldfappend = pd.DataFrame(mallist)

In [19]:
maldfappend = maldfappend.set_index('key')

In [21]:
maldfappend.columns = maldfappend.columns.str.replace(':', '')

In [29]:
maldfappend

,Title,Japanese,Type,Episodes,Status,Aired,Premiered,Broadcast,Producers,Licensors,Studios,Source,Genres,Duration,Rating,Score,Ranked,Popularity,Members,Favorites
key,,,,,,,,,,,,,,,,,,,,
39595,Megazone 23 XI,メガゾーン23 XI,TV,0,Not yet aired,Not available,?,Unknown,"None found,add some","None found,add some","None found,add some",Original,"Action,Mecha,Music,Mystery,Romance,Sci-Fi",Unknown,None,N/A1(scored by0users)1\n This score is ...,#022\n based on the top anime page. Please ...,#0,0,0


In [28]:
maldfappend['Episodes'] = maldfappend['Episodes'].replace({',',''})
maldfappend['Episodes'] = maldfappend['Episodes'].replace('Unknown', '0')
maldfappend.Episodes = maldfappend.Episodes.astype('int32')

In [33]:
maldfappend['Members'] = maldfappend['Members'].str.replace(',','').astype('int64')
maldfappend['Favorites'] = maldfappend['Favorites'].str.replace(',','').astype('int64')
maldfappend['ScoreExtract'] = maldfappend.Score.str.extract('(\d+.\d+)').astype('float64')

maldfappend['ScoreRaters'] = maldfappend.Score.str.extract('(?<=by)(.*)(?=users)')
maldfappend['ScoreRaters'] = maldfappend['ScoreRaters'].str.replace(',', '').astype('int64')

In [35]:
maldfupdate = pd.concat([maldf, maldfappend], sort=True)

In [44]:
maldfupdate.loc[39595]

Aired                                               Not available
Broadcast                                                 Unknown
Duration                                                  Unknown
English                                                       NaN
Episodes                                                        0
Favorites                                                       0
Genres                  Action,Mecha,Music,Mystery,Romance,Sci-Fi
Japanese                                               メガゾーン23 XI
Licensors                                     None found,add some
Members                                                         0
Popularity                                                     #0
Premiered                                                       ?
Producers                                     None found,add some
Ranked          #022\n    based on the top anime page. Please ...
Rating                                                       None
Score     

In [45]:
#maldfupdate.to_json(r'maldf.json', orient='index')

In [62]:
sequelkeys

[[39595, 'Prequel:', 1935], [39595, 'Prequel:', 39596]]

In [63]:
for i in sequelkeys:
    i[1] = i[1][:-1]

In [64]:
sequelkeys

[[39595, 'Prequel', 1935], [39595, 'Prequel', 39596]]

In [74]:
with open('sequelkeys.txt', 'r', encoding='utf8') as filehandle:  
    sequelkeysold = json.load(filehandle)

In [66]:
sequelkeysold[-10:]

[[40272, 'Prequel', 39184],
 [40280, 'Parent story', 38480],
 [40282, 'Alternative version', 11897],
 [40295, 'Spin-off', 38816],
 [40298, 'Adaptation', 88930],
 [40298, 'Adaptation', 98436],
 [40298, 'Other', 37055],
 [40301, 'Other', 1672],
 [40302, 'Other', 5023],
 [40302, 'Alternative version', 5232]]

In [67]:
sequelkeysupdate = sequelkeysold + sequelkeys

In [68]:
sequelkeysupdate[-10:]

[[40282, 'Alternative version', 11897],
 [40295, 'Spin-off', 38816],
 [40298, 'Adaptation', 88930],
 [40298, 'Adaptation', 98436],
 [40298, 'Other', 37055],
 [40301, 'Other', 1672],
 [40302, 'Other', 5023],
 [40302, 'Alternative version', 5232],
 [39595, 'Prequel', 1935],
 [39595, 'Prequel', 39596]]

In [69]:
with open('sequelkeys.txt', 'w', encoding='utf8') as filehandle:  
    json.dump(sequelkeysupdate, filehandle)

## Search sequel keys for missing values

In [4]:
with open(r'sequelkeys.txt', 'r', encoding='utf8') as filehandle:  
    sequelkeys = json.load(filehandle)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
sequeldf = pd.read_json(r'sequelkeys.json', orient='index')

In [8]:
sequeldf.sort_index(inplace=True)
sequeldf.head()

,PrimaryKey,RelationKey,RelationType
0,1,173,Adaptation
1,1,174,Adaptation
2,1,5,Side story
3,1,17205,Side story
4,1,4037,Summary


In [9]:
# sequeldf = pd.DataFrame(sequelkeys, columns = ['PrimaryKey', 'RelationType', 'RelationKey'])

In [9]:
sequelkeyset = set([x for row in 
                sequeldf.loc[sequeldf['RelationType'] != 'Adaptation'][['PrimaryKey', 'RelationKey']].values 
                for x in row])

In [10]:
sequelkeyset

{1,
 32772,
 5,
 6,
 32773,
 8,
 32777,
 32778,
 15,
 16,
 32785,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 32792,
 26,
 32794,
 32796,
 32797,
 30,
 31,
 32,
 33,
 32801,
 32802,
 32806,
 32807,
 32808,
 32809,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 32816,
 32817,
 53,
 54,
 32818,
 32820,
 32819,
 58,
 59,
 60,
 32826,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 32835,
 71,
 72,
 73,
 32836,
 75,
 76,
 77,
 32842,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 32866,
 101,
 32869,
 103,
 32870,
 32867,
 106,
 107,
 108,
 109,
 110,
 32875,
 112,
 32881,
 32876,
 115,
 116,
 117,
 118,
 32885,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 32898,
 130,
 131,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 32900,
 141,
 32901,
 143,
 144,
 32906,
 147,
 148,
 149,
 150,
 151,
 152,
 32916,
 154,
 155,
 156,
 157,
 158,
 32926,
 32927,
 161,
 32930,
 32924,
 165,
 32933,
 32935,
 168,
 169,
 170,
 32937,
 32934,
 32936,
 174,


In [14]:
maldfkeys = set(maldf.index)

In [15]:
maldfkeys

{32768,
 1,
 32772,
 5,
 6,
 7,
 8,
 32773,
 32775,
 32776,
 32777,
 32778,
 32779,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 32794,
 32796,
 32797,
 32799,
 32800,
 32801,
 32802,
 32803,
 32804,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 32829,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 32842,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 32899,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 32932,
 173,
 174,
 175

In [14]:
sequeldf.head()

,PrimaryKey,RelationType,RelationKey
0,1,Adaptation,173
1,1,Adaptation,174
2,1,Side story,5
3,1,Side story,17205
4,1,Summary,4037


In [15]:
x = sequeldf.loc[sequeldf['RelationType'] != 'Adaptation']

In [17]:
len(sequelkeyset)

9548

In [16]:
len(maldfkeys)

16019

In [19]:
missingsequels = sequelkeyset.difference(maldfkeys)

In [20]:
len(missingsequels)

11

In [21]:
missingsequels

{23207, 39393, 42749, 50937, 52007, 64261, 88923, 89715, 96221, 100274, 107935}

In [20]:
missingsequels = sorted(list(missingsequels))

In [21]:
missingsequels[0:10]

[39320, 39321, 39329, 39350, 39356, 39365, 39394, 39395, 39396, 39402]

In [34]:
i = missingsequels[0:2]
i

[39320, 39321]

In [36]:
i = missingsequels[0:2]
for idx, key in enumerate(i):
    print(str(key))

39320
39321


In [174]:
missingsequelsappend = []
failedlist = []
mallist = []
sequelkeys = []

maxindex = len(missingsequels)
minlooptime = 3
failedcount = 0

In [177]:
# get full url, update more
# i = missingsequels[0:2]

for idx, key in enumerate(missingsequels):
    # start timer
    timestart = time.time()
    
    try:
        url = 'https://myanimelist.net/anime/' + str(key)
        page = requests.get(url,timeout=5)
        page.raise_for_status()
    except requests.exceptions.RequestException as e:
        failedcount += 1 
        print(e)
        print('Current Failed Count: ' + str(failedcount))
        failedlist.append(url)
        
        if time.time() - timestart < minlooptime:
            time.sleep(minlooptime - (time.time() - timestart))
        continue
        
    newrow = {'key': key}

    # create soup
    anime = BeautifulSoup(page.content, 'html.parser')
    
    # get full url
    urlfull = anime.select('.horiznav_active')[0]['href']
    
    # append full url
    missingsequelsappend.append(urlfull)
    
    # get title
    title = {'Title': anime.select('.h1 span')[0].string}
    newrow.update(title)
    
    # get sidebar information
    sidebar = [a for a in 
               [[x for x in entry if x != '\n'] for entry in anime.select('.js-scrollfix-bottom div')]
               if len(a) > 1 and a[0].name == 'span']
    sidebardict = dict([entry[0].get_text(), tagstringextract(entry[1:])] for entry in sidebar)
    newrow.update(sidebardict)
    
    # add to global list
    mallist.append(newrow)
    
    # related
    relatedanime = anime.select('.anime_detail_related_anime .borderClass')
    relatedlist = [[key, alpha[0].text, y] for alpha in chunks(relatedanime, 2) for y in malkey(alpha[1])]
    
    sequelkeys += relatedlist
    
    # pause if too fast
    if time.time() - timestart < minlooptime:
        time.sleep(minlooptime - (time.time() - timestart))
    print('Processed anime [key = ' + str(key) + '], ' 
          + '%d' % (idx+1) + '/' + str(maxindex) + ', ' + '%.2f' % (100*(idx+1)/maxindex) + '%: in ' 
          + '%.4f' % (time.time() - timestart) + ' seconds.')


404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39350
Current Failed Count: 10
404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39402
Current Failed Count: 11
404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39447
Current Failed Count: 12
404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39630
Current Failed Count: 13
404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39651
Current Failed Count: 14
404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39746
Current Failed Count: 15
404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39768
Current Failed Count: 16
404 Client Error: Not Found for url: https://myanimelist.net/anime/https://myanimelist.net/anime/39932
C

KeyboardInterrupt: 

In [25]:
failedlist

NameError: name 'failedlist' is not defined

In [90]:
relatedanime[1]

<td class="borderClass" width="100%"><a href="/anime/33909/Marine_Dreamin">Marine Dreamin'</a></td>

## Update lists

In [188]:
len(animelist)

15976

In [199]:
animelist = animelist + missingsequelsappend

In [201]:
animelist.sort(key = malkey)

In [204]:
#with open('urllist.txt', 'w', encoding='utf8') as filehandle:  
#    json.dump(animelist, filehandle)

In [206]:
len(sequelkeys)

94

In [207]:
with open(r'sequelkeys.txt', 'r', encoding='utf8') as filehandle:  
    sequelkeysold = json.load(filehandle)

In [208]:
len(sequelkeysold)

22742

In [211]:
sequelkeysall = sequelkeysold + sequelkeys

In [212]:
len(sequelkeysall)

22836

In [216]:
failedlistkeys = set(malkey(failedlist))
failedlistkeys

{39350, 39402, 39447, 39630, 39651, 39746, 39768, 39932, 40021}

In [217]:
# filter out sequels pointing at invalid links
sequelkeysfiltered = [x for x in sequelkeysall if x[2] not in failedlistkeys]

In [218]:
len(sequelkeysfiltered)

22819

In [227]:
type(sequelkeysfiltered[0][2])

int

In [246]:
sequelkeysdf = pd.DataFrame(sequelkeysfiltered, columns = ['PrimaryKey', 'RelationType', 'RelationKey'])

In [247]:
sequelkeysdf.head()

,PrimaryKey,RelationType,RelationKey
0,1,Adaptation,173
1,1,Adaptation,174
2,1,Side story,5
3,1,Side story,17205
4,1,Summary,4037


In [68]:
sequelkeysdf.to_json(r'sequelkeys.json', orient='index')

NameError: name 'sequelkeysdf' is not defined

In [250]:
# maldf update

In [251]:
maldf.head()

,Aired,Broadcast,Duration,English,Episodes,Favorites,Genres,Japanese,Licensors,Members,...,Rating,Score,ScoreExtract,ScoreRaters,Source,Status,Studios,Synonyms,Title,Type
1,"Apr 3, 1998 to Apr 24, 1999",Saturdays at 01:00 (JST),24 min. per ep.,Cowboy Bebop,26,"46,627","Action,Adventure,Comedy,Drama,Sci-Fi,Space",カウボーイビバップ,"Funimation,Bandai Entertainment","864,330",...,R - 17+ (violence & profanity),"8.821(scored by435,335users)1\n indicat...",NaN,NaN,Original,Finished Airing,Sunrise,None,Cowboy Bebop,TV
5,"Sep 1, 2001",None,1 hr. 55 min.,Cowboy Bebop: The Movie,1,849,"Action,Drama,Mystery,Sci-Fi,Space",カウボーイビバップ 天国の扉,Sony Pictures Entertainment,"211,101",...,R - 17+ (violence & profanity),"8.411(scored by126,733users)1\n indicat...",NaN,NaN,Original,Finished Airing,Bones,Cowboy Bebop: Knockin' on Heaven's Door,Cowboy Bebop: Tengoku no Tobira,Movie
6,"Apr 1, 1998 to Sep 30, 1998",Thursdays at 01:15 (JST),24 min. per ep.,Trigun,26,"10,919","Action,Sci-Fi,Adventure,Comedy,Drama,Shounen",トライガン,"Funimation,Geneon Entertainment USA","434,992",...,PG-13 - Teens 13 or older,"8.291(scored by222,984users)1\n indicat...",NaN,NaN,Manga,Finished Airing,Madhouse,None,Trigun,TV
7,"Jul 2, 2002 to Dec 24, 2002",Tuesdays at Unknown,25 min. per ep.,Witch Hunter Robin,26,548,"Action,Magic,Police,Supernatural,Drama,Mystery",Witch Hunter ROBIN,"Funimation,Bandai Entertainment","82,627",...,PG-13 - Teens 13 or older,"7.321(scored by33,573users)1\n indicate...",NaN,NaN,Original,Finished Airing,Sunrise,WHR,Witch Hunter Robin,TV
8,"Sep 30, 2004 to Sep 29, 2005",Thursdays at 18:30 (JST),23 min. per ep.,Beet the Vandel Buster,52,15,"Adventure,Fantasy,Shounen,Supernatural",冒険王ビィト,"None found,add some","12,055",...,PG - Children,"7.031(scored by4,996users)1\n indicates...",NaN,NaN,Manga,Finished Airing,Toei Animation,Adventure King Beet,Bouken Ou Beet,TV


In [259]:
len(maldf.columns)

24

In [254]:
maldfnew = pd.DataFrame(mallist)

In [257]:
maldfnew = maldfnew.set_index('key')
maldfnew.columns = maldfnew.columns.str.replace(':', '')

maldfnew['Episodes'] = maldfnew['Episodes'].replace({',',''})
maldfnew['Episodes'] = maldfnew['Episodes'].replace('Unknown', '0')
maldfnew.Episodes = maldfnew.Episodes.astype('int32')

maldfnew['Members'] = maldfnew['Members'].str.replace(',','').astype('int64')
maldfnew['Favorites'] = maldfnew['Favorites'].str.replace(',','').astype('int64')
maldfnew['ScoreExtract'] = maldfnew.Score.str.extract('(\d+.\d+)').astype('float64')

maldfnew['ScoreRaters'] = maldfnew.Score.str.extract('(?<=by)(.*)(?=users)')
maldfnew['ScoreRaters'] = maldfnew['ScoreRaters'].str.replace(',', '').astype('int64')



In [258]:
maldfnew.head()

,Title,Japanese,Type,Episodes,Status,Aired,Producers,Licensors,Studios,Source,...,Ranked,Popularity,Members,Favorites,Synonyms,English,Premiered,Broadcast,ScoreExtract,ScoreRaters
key,,,,,,,,,,,,,,,,,,,,,
39320,Marine Dreamin' (2017ver),Marine Dreamin' (MUSIC VIDEO 2017ver),Music,1,Finished Airing,"Apr 28, 2017","None found,add some","None found,add some","None found,add some",Game,...,#022\n based on the top anime page. Please ...,#0,0,0,NaN,NaN,NaN,NaN,NaN,0
39321,Dive to Blue (MMD ver),Dive to Blue (MMD MUSIC VIDEO),Music,1,Finished Airing,"Oct 20, 2016","None found,add some","None found,add some","None found,add some",Game,...,#022\n based on the top anime page. Please ...,#0,0,0,NaN,NaN,NaN,NaN,NaN,0
39329,Tenkuu no Escaflowne Recaps,天空のエスカフローネ BD-BOX 特別編集版,Special,4,Finished Airing,"Feb 24, 2012","None found,add some","None found,add some",Sunrise,Original,...,#724022\n based on the top anime page. Plea...,#10177,660,2,"The Vision of Escaflowne Recaps, Escaflowne BD...",NaN,NaN,NaN,6.251,67
39356,Neko Neko Nihonshi 4th Season,ねこねこ日本史 第4期,TV,0,Currently Airing,"Apr 3, 2019 to ?","None found,add some","None found,add some","None found,add some",Web manga,...,#1300122\n based on the top anime page. Ple...,#15830,53,0,NaN,Meow Meow Japanese History,Spring 2019,Unknown,6.001,1
39365,Ero Konbini Tenchou,エロコンビニ店長,OVA,0,Currently Airing,"Feb 22, 2019 to ?","None found,add some","None found,add some",PoRO,Visual novel,...,N/A22\n based on the top anime page. Please...,#9864,757,11,NaN,NaN,NaN,NaN,6.491,113


In [260]:
len(maldfnew.columns)

24

In [261]:
maldfupdate = pd.concat([maldf, maldfnew], sort=True)

In [262]:
maldfupdate.to_json(r'maldf.json', orient='index')

In [26]:
# remove invalid keys from sequelkeys
failedlist = missingsequels

In [30]:
sequeldf.head()

,PrimaryKey,RelationKey,RelationType
0,1,173,Adaptation
1,1,174,Adaptation
2,1,5,Side story
3,1,17205,Side story
4,1,4037,Summary


In [59]:
sequeldf['RelationType'] = sequeldf.RelationType.str.replace(':','')

In [69]:
sequeldffilter2 = sequeldf[~(sequeldf.RelationKey.isin(failedlist)) & 
                                 (sequeldf.RelationType != 'Adaptation')]

In [67]:
failedlist

{23207, 39393, 42749, 50937, 52007, 64261, 88923, 89715, 96221, 100274, 107935}

In [70]:
sequeldffilter2

,PrimaryKey,RelationKey,RelationType
2,1,5,Side story
3,1,17205,Side story
4,1,4037,Summary
5,5,1,Parent story
7,6,4106,Side story
...,...,...,...
22808,40062,39080,Prequel
22809,40063,15407,Other
22815,40073,16157,Full story
22816,40076,37141,Other


In [35]:
len(sequeldf)

22819

In [45]:
len(sequeldffilter)

13

In [46]:
failedlist

{23207, 39393, 42749, 50937, 52007, 64261, 88923, 89715, 96221, 100274, 107935}

In [49]:
sequeldffilter

,PrimaryKey,RelationKey,RelationType
12044,16157,23207,Adaptation
12045,16157,42749,Adaptation
12046,16157,50937,Adaptation
12047,16157,52007,Adaptation
12048,16157,64261,Adaptation
17183,32572,96221,Adaptation
19209,35302,96221,Adaptation
20298,36882,88923,Adaptation
20697,37369,96221,Adaptation
21076,37940,107935,Adaptation


In [71]:
#sequeldffilter2.to_json(r'sequelkeys.json', orient='index')

1

In [33]:
'https://myanimelist.net/anime/' + str(i)

'https://myanimelist.net/anime/39320'

In [35]:
url = 'https://myanimelist.net/anime/' + str(i)
page = requests.get(url,timeout=5)
page.raise_for_status()

In [38]:
anime = BeautifulSoup(page.content, 'html.parser')

In [47]:
anime.select('.horiznav_active')[0]['href']

'https://myanimelist.net/anime/39320/Marine_Dreamin_2017ver'

In [118]:
sequeldf.loc[sequeldf['RelationKey'] == 39350]

,PrimaryKey,RelationType,RelationKey
21395,38196,Sequel,39350


### Implement binary insert later for optimization

In [40]:
testsort = ['https://myanimelist.net/anime/40298/Youjo_Senki_Movie__Manner_Eizou',
            'https://myanimelist.net/anime/40301/Bomberman__Yuuki_wo_Arigatou_-_Watashi_ga_Mimi_ni_Naru',
            'https://myanimelist.net/anime/99/Mai-Otome',
            'https://myanimelist.net/anime/40299/Gokiburi-chan',
            'https://myanimelist.net/anime/1/Cowboy_Bebop']

In [18]:
testsort.sort()

In [19]:
testsort

['https://myanimelist.net/anime/1/Cowboy_Bebop',
 'https://myanimelist.net/anime/40298/Youjo_Senki_Movie__Manner_Eizou',
 'https://myanimelist.net/anime/40299/Gokiburi-chan',
 'https://myanimelist.net/anime/40301/Bomberman__Yuuki_wo_Arigatou_-_Watashi_ga_Mimi_ni_Naru',
 'https://myanimelist.net/anime/99/Mai-Otome']

In [57]:
testsort.sort(key=malkey)

In [22]:
testsort

['https://myanimelist.net/anime/1/Cowboy_Bebop',
 'https://myanimelist.net/anime/99/Mai-Otome',
 'https://myanimelist.net/anime/40298/Youjo_Senki_Movie__Manner_Eizou',
 'https://myanimelist.net/anime/40299/Gokiburi-chan',
 'https://myanimelist.net/anime/40301/Bomberman__Yuuki_wo_Arigatou_-_Watashi_ga_Mimi_ni_Naru']

In [66]:
inserturl = 'https://myanimelist.net/anime/39595/Megazone_23_XI'

In [35]:
[i for i in testsort if malkey(i) < malkey(inserturl)]

['https://myanimelist.net/anime/1/Cowboy_Bebop',
 'https://myanimelist.net/anime/99/Mai-Otome']

In [37]:
insertindex = len([i for i in testsort if malkey(i) < malkey(inserturl)])

2

In [44]:
testsort.insert(len([i for i in testsort if malkey(i) < malkey(inserturl)]),
                inserturl)

In [45]:
testsort

['https://myanimelist.net/anime/1/Cowboy_Bebop',
 'https://myanimelist.net/anime/99/Mai-Otome',
 'https://myanimelist.net/anime/39595/Megazone_23_XI',
 'https://myanimelist.net/anime/40298/Youjo_Senki_Movie__Manner_Eizou',
 'https://myanimelist.net/anime/40299/Gokiburi-chan',
 'https://myanimelist.net/anime/40301/Bomberman__Yuuki_wo_Arigatou_-_Watashi_ga_Mimi_ni_Naru']

In [65]:
next(x for x, val for i in testsort if malkey(i) > malkey(inserturl))

'https://myanimelist.net/anime/40298/Youjo_Senki_Movie__Manner_Eizou'

In [55]:
import bisect

In [58]:
testsort

['https://myanimelist.net/anime/1/Cowboy_Bebop',
 'https://myanimelist.net/anime/99/Mai-Otome',
 'https://myanimelist.net/anime/39595/Megazone_23_XI',
 'https://myanimelist.net/anime/40298/Youjo_Senki_Movie__Manner_Eizou',
 'https://myanimelist.net/anime/40299/Gokiburi-chan',
 'https://myanimelist.net/anime/40301/Bomberman__Yuuki_wo_Arigatou_-_Watashi_ga_Mimi_ni_Naru']